In [1]:
import re
import torch.nn as nn
import numpy as np 
import pandas as pd
import os
import tiktoken
from torch.utils.data import Dataset, DataLoader
import torch
import math
import time
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
#Tokenizer
import tiktoken
tokenizer =  tiktoken.get_encoding("gpt2")

In [3]:
def text_to_tokens(text):
    text = tokenizer.encode(text)
    token = torch.tensor(text).unsqueeze(0)
    return token
def tokens_to_text(tokens):
    return tokenizer.decode(tokens.squeeze(0).tolist())

In [4]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        tokens = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(tokens) - max_length, stride):
            input_chuck = tokens[i:i+max_length]
            target_chuck = tokens[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chuck))
            self.target_ids.append(torch.tensor(target_chuck)) 

    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

In [5]:
LLM_Config = {
    "vocab_size": 50257, #All Words in LLM
    'context_length': 256, #Size of the each row in batch
    'emb_dim': 768, #Dimensions of Embeddinging Vectors
    "n_heads": 12, #Number of Transfoemrs
    "n_layers": 12, #Number of Transformer layers
    "drop_rate": 0.1, #Percentage of Tokens that are dropped
    "qkv_bias": False
}

In [6]:
class Causual_attentionV1(torch.nn.Module):
    def __init__(self, d_in, d_out, context_len, dropout, bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = torch.nn.Linear(d_in, d_out, bias=bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=bias)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_len,context_len), diagonal=1))

    def forward(self, x):
        b, nums_t, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_score = queries @ keys.transpose(1,2)
        attn_score.masked_fill(self.mask.bool()[:nums_t, :nums_t], -torch.inf)
        attn_weights = torch.softmax(attn_score/(keys.shape[-1]**0.5), dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec

class multiheadAttentionV1(torch.nn.Module):
    def __init__(self, d_in, d_out, context_len, dropout, heads, bias=False):
        super().__init__()
        self.heads = torch.nn.ModuleList([Causual_attentionV1(d_in, d_out, context_len, dropout, bias) for _ in range(heads)])
        self.out_proj = nn.Linear(d_out * heads, d_out)
        
    def forward(self, x):
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        return self.out_proj(x)
        

In [7]:
import torch.nn as nn
class GPT(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['drop_rate'])
        self.trf_block = nn.Sequential(*[TransformerBlock (cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg['emb_dim'], cfg['vocab_size'], bias=False)
    
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x= self.drop_emb(x)
        x= self.trf_block(x)
        x= self.final_norm(x)
        logits = self.out_head(x)
        return logits

class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.tensor(2/torch.pi))  * (x + 0.044715 * x ** 3))

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = multiheadAttentionV1(
            d_in=cfg['emb_dim'],
            d_out=cfg['emb_dim'],
            context_len=cfg['context_length'],
            dropout=cfg['drop_rate'],
            heads=cfg['n_heads'],
            bias=cfg['qkv_bias']
        )

        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg['drop_rate'])
    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x= self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x= self.drop_shortcut(x)
        x = x + shortcut 
        

        return x

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(cfg['emb_dim'], 4*cfg['emb_dim']), GELU(), nn.Linear(4* cfg['emb_dim'],cfg['emb_dim']))

    def forward(self, x):
        return self.layers(x)

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, unbiased=False, keepdim=True)
        norm_x = (x - mean)/(torch.sqrt( var+self.eps ))
        return self.scale * norm_x + self.shift

In [8]:
class DNN(nn.Module):
    def __init__(self, layers, shortcut):
        super().__init__()
        self.shortcut =shortcut
        self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layers[0],layers[1]), GELU()),
            nn.Sequential(nn.Linear(layers[1],layers[2]), GELU()),
            nn.Sequential(nn.Linear(layers[2],layers[3]), GELU()),
            nn.Sequential(nn.Linear(layers[3],layers[4]), GELU()),
            nn.Sequential(nn.Linear(layers[4],layers[5]), GELU())
        ])
    def forward(self, x):
        for layer in self.layers:
            output = layer(x)
            if self.shortcut and x.shape == output.shape:
                x = x+output
            else:
                x = output
        return x

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
def gen_text(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1,:]
        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [11]:
def batch_loss(inputs, targets, model, device):
    inputs, targets = inputs.to(device), targets.to(device)
    logits = model(inputs)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0,1), targets.flatten())
    return loss

def calc_loss_loader(data_loader, model, device, num=None):
    total = 0
    if len(data_loader) == 0:
        return float('nan')
    elif num is None:
        num = len(data_loader)
    else:
        num = min(num, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num:
            loss = batch_loss(input_batch, target_batch, model, device)
            total += loss.item()
        else:
            break
    return total/num

In [12]:
def gen_print_sample(model, tokenizer, device, start_context):
    model.eval()
    cs = model.pos_emb.weight.shape[0]
    encoded = text_to_tokens(start_context).to(device)
    with torch.no_grad():
        token_ids = gen_text(model, encoded, 50, cs)
    text = tokens_to_text(token_ids)
    print(text.replace('\n', ' '))
    model.train()
    

In [13]:
verdict_story = """I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it's going to send the value of my picture 'way up; but I don't think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing's lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn's "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?

Well!--even through the prism of Hermia's tears I felt able to face the fact with equanimity. Poor Jack Gisburn! The women had made him--it was fitting that they should mourn him. Among his own sex fewer regrets were heard, and in his own trade hardly a murmur. Professional jealousy? Perhaps. If it were, the honour of the craft was vindicated by little Claude Nutley, who, in all good faith, brought out in the Burlington a very handsome "obituary" on Jack--one of those showy articles stocked with random technicalities that I have heard (I won't say by whom) compared to Gisburn's painting. And so--his resolve being apparently irrevocable--the discussion gradually died out, and, as Mrs. Thwing had predicted, the price of "Gisburns" went up.

It was not till three years later that, in the course of a few weeks' idling on the Riviera, it suddenly occurred to me to wonder why Gisburn had given up his painting. On reflection, it really was a tempting problem. To accuse his wife would have been too easy--his fair sitters had been denied the solace of saying that Mrs. Gisburn had "dragged him down." For Mrs. Gisburn--as such--had not existed till nearly a year after Jack's resolve had been taken. It might be that he had married her--since he liked his ease--because he didn't want to go on painting; but it would have been hard to prove that he had given up his painting because he had married her.

Of course, if she had not dragged him down, she had equally, as Miss Croft contended, failed to "lift him up"--she had not led him back to the easel. To put the brush into his hand again--what a vocation for a wife! But Mrs. Gisburn appeared to have disdained it--and I felt it might be interesting to find out why.

The desultory life of the Riviera lends itself to such purely academic speculations; and having, on my way to Monte Carlo, caught a glimpse of Jack's balustraded terraces between the pines, I had myself borne thither the next day.

I found the couple at tea beneath their palm-trees; and Mrs. Gisburn's welcome was so genial that, in the ensuing weeks, I claimed it frequently. It was not that my hostess was "interesting": on that point I could have given Miss Croft the fullest reassurance. It was just because she was _not_ interesting--if I may be pardoned the bull--that I found her so. For Jack, all his life, had been surrounded by interesting women: they had fostered his art, it had been reared in the hot-house of their adulation. And it was therefore instructive to note what effect the "deadening atmosphere of mediocrity" (I quote Miss Croft) was having on him.

I have mentioned that Mrs. Gisburn was rich; and it was immediately perceptible that her husband was extracting from this circumstance a delicate but substantial satisfaction. It is, as a rule, the people who scorn money who get most out of it; and Jack's elegant disdain of his wife's big balance enabled him, with an appearance of perfect good-breeding, to transmute it into objects of art and luxury. To the latter, I must add, he remained relatively indifferent; but he was buying Renaissance bronzes and eighteenth-century pictures with a discrimination that bespoke the amplest resources.

"Money's only excuse is to put beauty into circulation," was one of the axioms he laid down across the Sevres and silver of an exquisitely appointed luncheon-table, when, on a later day, I had again run over from Monte Carlo; and Mrs. Gisburn, beaming on him, added for my enlightenment: "Jack is so morbidly sensitive to every form of beauty."

Poor Jack! It had always been his fate to have women say such things of him: the fact should be set down in extenuation. What struck me now was that, for the first time, he resented the tone. I had seen him, so often, basking under similar tributes--was it the conjugal note that robbed them of their savour? No--for, oddly enough, it became apparent that he was fond of Mrs. Gisburn--fond enough not to see her absurdity. It was his own absurdity he seemed to be wincing under--his own attitude as an object for garlands and incense.

"My dear, since I've chucked painting people don't say that stuff about me--they say it about Victor Grindle," was his only protest, as he rose from the table and strolled out onto the sunlit terrace.

I glanced after him, struck by his last word. Victor Grindle was, in fact, becoming the man of the moment--as Jack himself, one might put it, had been the man of the hour. The younger artist was said to have formed himself at my friend's feet, and I wondered if a tinge of jealousy underlay the latter's mysterious abdication. But no--for it was not till after that event that the _rose Dubarry_ drawing-rooms had begun to display their "Grindles."

I turned to Mrs. Gisburn, who had lingered to give a lump of sugar to her spaniel in the dining-room.

"Why _has_ he chucked painting?" I asked abruptly.

She raised her eyebrows with a hint of good-humoured surprise.

"Oh, he doesn't _have_ to now, you know; and I want him to enjoy himself," she said quite simply.

I looked about the spacious white-panelled room, with its _famille-verte_ vases repeating the tones of the pale damask curtains, and its eighteenth-century pastels in delicate faded frames.

"Has he chucked his pictures too? I haven't seen a single one in the house."

A slight shade of constraint crossed Mrs. Gisburn's open countenance. "It's his ridiculous modesty, you know. He says they're not fit to have about; he's sent them all away except one--my portrait--and that I have to keep upstairs."

His ridiculous modesty--Jack's modesty about his pictures? My curiosity was growing like the bean-stalk. I said persuasively to my hostess: "I must really see your portrait, you know."

She glanced out almost timorously at the terrace where her husband, lounging in a hooded chair, had lit a cigar and drawn the Russian deerhound's head between his knees.

"Well, come while he's not looking," she said, with a laugh that tried to hide her nervousness; and I followed her between the marble Emperors of the hall, and up the wide stairs with terra-cotta nymphs poised among flowers at each landing.

In the dimmest corner of her boudoir, amid a profusion of delicate and distinguished objects, hung one of the familiar oval canvases, in the inevitable garlanded frame. The mere outline of the frame called up all Gisburn's past!

Mrs. Gisburn drew back the window-curtains, moved aside a _jardiniere_ full of pink azaleas, pushed an arm-chair away, and said: "If you stand here you can just manage to see it. I had it over the mantel-piece, but he wouldn't let it stay."

Yes--I could just manage to see it--the first portrait of Jack's I had ever had to strain my eyes over! Usually they had the place of honour--say the central panel in a pale yellow or _rose Dubarry_ drawing-room, or a monumental easel placed so that it took the light through curtains of old Venetian point. The more modest place became the picture better; yet, as my eyes grew accustomed to the half-light, all the characteristic qualities came out--all the hesitations disguised as audacities, the tricks of prestidigitation by which, with such consummate skill, he managed to divert attention from the real business of the picture to some pretty irrelevance of detail. Mrs. Gisburn, presenting a neutral surface to work on--forming, as it were, so inevitably the background of her own picture--had lent herself in an unusual degree to the display of this false virtuosity. The picture was one of Jack's "strongest," as his admirers would have put it--it represented, on his part, a swelling of muscles, a congesting of veins, a balancing, straddling and straining, that reminded one of the circus-clown's ironic efforts to lift a feather. It met, in short, at every point the demand of lovely woman to be painted "strongly" because she was tired of being painted "sweetly"--and yet not to lose an atom of the sweetness.

"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride. "The last but one," she corrected herself--"but the other doesn't count, because he destroyed it."

"Destroyed it?" I was about to follow up this clue when I heard a footstep and saw Jack himself on the threshold.

As he stood there, his hands in the pockets of his velveteen coat, the thin brown waves of hair pushed back from his white forehead, his lean sunburnt cheeks furrowed by a smile that lifted the tips of a self-confident moustache, I felt to what a degree he had the same quality as his pictures--the quality of looking cleverer than he was.

His wife glanced at him deprecatingly, but his eyes travelled past her to the portrait.

"Mr. Rickham wanted to see it," she began, as if excusing herself. He shrugged his shoulders, still smiling.

"Oh, Rickham found me out long ago," he said lightly; then, passing his arm through mine: "Come and see the rest of the house."

He showed it to me with a kind of naive suburban pride: the bath-rooms, the speaking-tubes, the dress-closets, the trouser-presses--all the complex simplifications of the millionaire's domestic economy. And whenever my wonder paid the expected tribute he said, throwing out his chest a little: "Yes, I really don't see how people manage to live without that."

Well--it was just the end one might have foreseen for him. Only he was, through it all and in spite of it all--as he had been through, and in spite of, his pictures--so handsome, so charming, so disarming, that one longed to cry out: "Be dissatisfied with your leisure!" as once one had longed to say: "Be dissatisfied with your work!"

But, with the cry on my lips, my diagnosis suffered an unexpected check.

"This is my own lair," he said, leading me into a dark plain room at the end of the florid vista. It was square and brown and leathery: no "effects"; no bric-a-brac, none of the air of posing for reproduction in a picture weekly--above all, no least sign of ever having been used as a studio.

The fact brought home to me the absolute finality of Jack's break with his old life.

"Don't you ever dabble with paint any more?" I asked, still looking about for a trace of such activity.

"Never," he said briefly.

"Or water-colour--or etching?"

His confident eyes grew dim, and his cheeks paled a little under their handsome sunburn.

"Never think of it, my dear fellow--any more than if I'd never touched a brush."

And his tone told me in a flash that he never thought of anything else.

I moved away, instinctively embarrassed by my unexpected discovery; and as I turned, my eye fell on a small picture above the mantel-piece--the only object breaking the plain oak panelling of the room.

"Oh, by Jove!" I said.

It was a sketch of a donkey--an old tired donkey, standing in the rain under a wall.

"By Jove--a Stroud!" I cried.

He was silent; but I felt him close behind me, breathing a little quickly.

"What a wonder! Made with a dozen lines--but on everlasting foundations. You lucky chap, where did you get it?"

He answered slowly: "Mrs. Stroud gave it to me."

"Ah--I didn't know you even knew the Strouds. He was such an inflexible hermit."

"I didn't--till after. . . . She sent for me to paint him when he was dead."

"When he was dead? You?"

I must have let a little too much amazement escape through my surprise, for he answered with a deprecating laugh: "Yes--she's an awful simpleton, you know, Mrs. Stroud. Her only idea was to have him done by a fashionable painter--ah, poor Stroud! She thought it the surest way of proclaiming his greatness--of forcing it on a purblind public. And at the moment I was _the_ fashionable painter."

"Ah, poor Stroud--as you say. Was _that_ his history?"

"That was his history. She believed in him, gloried in him--or thought she did. But she couldn't bear not to have all the drawing-rooms with her. She couldn't bear the fact that, on varnishing days, one could always get near enough to see his pictures. Poor woman! She's just a fragment groping for other fragments. Stroud is the only whole I ever knew."

"You ever knew? But you just said--"

Gisburn had a curious smile in his eyes.

"Oh, I knew him, and he knew me--only it happened after he was dead."

I dropped my voice instinctively. "When she sent for you?"

"Yes--quite insensible to the irony. She wanted him vindicated--and by me!"

He laughed again, and threw back his head to look up at the sketch of the donkey. "There were days when I couldn't look at that thing--couldn't face it. But I forced myself to put it here; and now it's cured me--cured me. That's the reason why I don't dabble any more, my dear Rickham; or rather Stroud himself is the reason."

For the first time my idle curiosity about my companion turned into a serious desire to understand him better.

"I wish you'd tell me how it happened," I said.

He stood looking up at the sketch, and twirling between his fingers a cigarette he had forgotten to light. Suddenly he turned toward me.

"I'd rather like to tell you--because I've always suspected you of loathing my work."

I made a deprecating gesture, which he negatived with a good-humoured shrug.

"Oh, I didn't care a straw when I believed in myself--and now it's an added tie between us!"

He laughed slightly, without bitterness, and pushed one of the deep arm-chairs forward. "There: make yourself comfortable--and here are the cigars you like."

He placed them at my elbow and continued to wander up and down the room, stopping now and then beneath the picture.

"How it happened? I can tell you in five minutes--and it didn't take much longer to happen. . . . I can remember now how surprised and pleased I was when I got Mrs. Stroud's note. Of course, deep down, I had always _felt_ there was no one like him--only I had gone with the stream, echoed the usual platitudes about him, till I half got to think he was a failure, one of the kind that are left behind. By Jove, and he _was_ left behind--because he had come to stay! The rest of us had to let ourselves be swept along or go under, but he was high above the current--on everlasting foundations, as you say.

"Well, I went off to the house in my most egregious mood--rather moved, Lord forgive me, at the pathos of poor Stroud's career of failure being crowned by the glory of my painting him! Of course I meant to do the picture for nothing--I told Mrs. Stroud so when she began to stammer something about her poverty. I remember getting off a prodigious phrase about the honour being _mine_--oh, I was princely, my dear Rickham! I was posing to myself like one of my own sitters.

"Then I was taken up and left alone with him. I had sent all my traps in advance, and I had only to set up the easel and get to work. He had been dead only twenty-four hours, and he died suddenly, of heart disease, so that there had been no preliminary work of destruction--his face was clear and untouched. I had met him once or twice, years before, and thought him insignificant and dingy. Now I saw that he was superb.

"I was glad at first, with a merely aesthetic satisfaction: glad to have my hand on such a 'subject.' Then his strange life-likeness began to affect me queerly--as I blocked the head in I felt as if he were watching me do it. The sensation was followed by the thought: if he _were_ watching me, what would he say to my way of working? My strokes began to go a little wild--I felt nervous and uncertain.

"Once, when I looked up, I seemed to see a smile behind his close grayish beard--as if he had the secret, and were amusing himself by holding it back from me. That exasperated me still more. The secret? Why, I had a secret worth twenty of his! I dashed at the canvas furiously, and tried some of my bravura tricks. But they failed me, they crumbled. I saw that he wasn't watching the showy bits--I couldn't distract his attention; he just kept his eyes on the hard passages between. Those were the ones I had always shirked, or covered up with some lying paint. And how he saw through my lies!

"I looked up again, and caught sight of that sketch of the donkey hanging on the wall near his bed. His wife told me afterward it was the last thing he had done--just a note taken with a shaking hand, when he was down in Devonshire recovering from a previous heart attack. Just a note! But it tells his whole history. There are years of patient scornful persistence in every line. A man who had swum with the current could never have learned that mighty up-stream stroke. . . .

"I turned back to my work, and went on groping and muddling; then I looked at the donkey again. I saw that, when Stroud laid in the first stroke, he knew just what the end would be. He had possessed his subject, absorbed it, recreated it. When had I done that with any of my things? They hadn't been born of me--I had just adopted them. . . .

"Hang it, Rickham, with that face watching me I couldn't do another stroke. The plain truth was, I didn't know where to put it--_I had never known_. Only, with my sitters and my public, a showy splash of colour covered up the fact--I just threw paint into their faces. . . . Well, paint was the one medium those dead eyes could see through--see straight to the tottering foundations underneath. Don't you know how, in talking a foreign language, even fluently, one says half the time not what one wants to but what one can? Well--that was the way I painted; and as he lay there and watched me, the thing they called my 'technique' collapsed like a house of cards. He didn't sneer, you understand, poor Stroud--he just lay there quietly watching, and on his lips, through the gray beard, I seemed to hear the question: 'Are you sure you know where you're coming out?'

"If I could have painted that face, with that question on it, I should have done a great thing. The next greatest thing was to see that I couldn't--and that grace was given me. But, oh, at that minute, Rickham, was there anything on earth I wouldn't have given to have Stroud alive before me, and to hear him say: 'It's not too late--I'll show you how'?

"It _was_ too late--it would have been, even if he'd been alive. I packed up my traps, and went down and told Mrs. Stroud. Of course I didn't tell her _that_--it would have been Greek to her. I simply said I couldn't paint him, that I was too moved. She rather liked the idea--she's so romantic! It was that that made her give me the donkey. But she was terribly upset at not getting the portrait--she did so want him 'done' by some one showy! At first I was afraid she wouldn't let me off--and at my wits' end I suggested Grindle. Yes, it was I who started Grindle: I told Mrs. Stroud he was the 'coming' man, and she told somebody else, and so it got to be true. . . . And he painted Stroud without wincing; and she hung the picture among her husband's things. . . ."

He flung himself down in the arm-chair near mine, laid back his head, and clasping his arms beneath it, looked up at the picture above the chimney-piece.

"I like to fancy that Stroud himself would have given it to me, if he'd been able to say what he thought that day."

And, in answer to a question I put half-mechanically--"Begin again?" he flashed out. "When the one thing that brings me anywhere near him is that I knew enough to leave off?"

He stood up and laid his hand on my shoulder with a laugh. "Only the irony of it is that I _am_ still painting--since Grindle's doing it for me! The Strouds stand alone, and happen once--but there's no exterminating our kind of art."""

In [14]:
torch.manual_seed(123)
train_ratio = int(0.90 * len(verdict_story))
train = verdict_story[:train_ratio]
test = verdict_story[train_ratio:]

train_loader = create_dataloader_v1(train, 2, LLM_Config['context_length'],LLM_Config['context_length'], True, True, 0 )
test_loader = create_dataloader_v1(test, 2, LLM_Config['context_length'],LLM_Config['context_length'], False, False, 0 )

In [15]:
def training_cycle(model, train_loader, test_loader, optimizer, device, num_epochs, eval_freq, eval_iter, start_context, tokenizer):
    train_lossess, test_losses, track_tokens = [],[],[]
    tokens_seen, gloabal_step = 0, -1

    for epoch in range(num_epochs):
        model.train()

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            loss = batch_loss(inputs, targets, model, device)
            loss.backward()
            optimizer.step()
            tokens_seen += inputs.numel()
            gloabal_step += 1

            if gloabal_step % eval_freq == 0:
                training_loss, test_loss = evalm(model, train_loader, test_loader, device, eval_iter)
                train_lossess.append(training_loss)
                test_losses.append(test_loss)
                track_tokens.append(tokens_seen)
                print(f"Epoch: {epoch+1}, Step {gloabal_step}:\n Train_loss:{training_loss}, Test_loss: {test_loss}")

        gen_print_sample(model, tokenizer, device, start_context)

    return train_lossess, test_losses, tokens_seen

In [16]:
def evalm(model, train_loader, test_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device)
        test_loss = calc_loss_loader(test_loader, model, device)
    model.train()
    return train_loss, test_loss

In [17]:
import time

start_time = time.time()

torch.manual_seed(123)
model = GPT(LLM_Config)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_lossess, tokens_seen = training_cycle(model, train_loader, test_loader, optimizer, device, num_epochs=num_epochs, eval_freq=5,eval_iter=5, 
                                                        start_context="Every effort moves you", tokenizer =tokenizer)
end_time = time.time()
print(f"Run Time: {end_time - start_time}")

Epoch: 1, Step 0:
 Train_loss:9.836820814344618, Test_loss: 9.990671157836914
Epoch: 1, Step 5:
 Train_loss:8.281566407945421, Test_loss: 8.464688301086426
Every effort moves you..................................................
Epoch: 2, Step 10:
 Train_loss:6.878113428751628, Test_loss: 7.168522357940674
Epoch: 2, Step 15:
 Train_loss:17.224855846828884, Test_loss: 16.87398910522461
Every effort moves you Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm Hamm
Epoch: 3, Step 20:
 Train_loss:6.2556849585639105, Test_loss: 6.687835693359375
Epoch: 3, Step 25:
 Train_loss:6.154180314805773, Test_loss: 6.612873077392578
Every effort moves you" was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was was 